In [10]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import earthpy as et
import fiona
import numpy as np
from shapely import wkt
import folium
#fiona.supported_drivers

In [11]:
os.chdir(os.path.join(et.io.HOME, 'mahbub-dev/beuth_hochschule_for_technik_berlin/berlin_datathon_2021/data'))

In [12]:
kfz_neukölln_02_path = os.path.join("kfz_neukölln_02.csv")
df_kfz_neukölln_02 = pd.read_csv(kfz_neukölln_02_path)
df_kfz_neukölln_02['geometry'] = df_kfz_neukölln_02['geometry'].apply(wkt.loads)
df_kfz_neukölln_02 = gpd.GeoDataFrame(df_kfz_neukölln_02, crs='epsg:4326')
df_kfz_neukölln_02.dtypes

lor                                object
name                               object
capacity                            int64
highway                            object
parking                            object
orientation                        object
position                           object
district                           object
Bezirksregion                      object
Prognoseraum                       object
lor size in m²                    float64
inhabitants_total                   int64
of_those_inhabitants_18+            int64
vehicles_overall                    int64
cars_only                           int64
vehicles_per_1000_inhabitants       int64
cars_per_1000_inhabitants           int64
geometry                         geometry
dtype: object

In [20]:
series_lor_parking_spots = df_kfz_neukölln_02.groupby(by=["lor"])["capacity"].sum()
df_lor_data = df_kfz_neukölln_02.groupby(by=["lor"])[["lor size in m²", "inhabitants_total", "vehicles_overall", "geometry"]].first()
df_lor_data["centroid"] = df_lor_data.centroid
df_lor_data["capacity"] = series_lor_parking_spots.values
df_lor_data = df_lor_data.reset_index()

df_lor_data = df_lor_data.sort_values("capacity", ascending=False)
df_lor_data.dtypes

lor                    object
lor size in m²        float64
inhabitants_total       int64
vehicles_overall        int64
geometry             geometry
centroid             geometry
capacity                int64
dtype: object

In [21]:
df_lor_data[["capacity", "vehicles_overall"]].describe()

,capacity,vehicles_overall
count,46.000000,46.000000
mean,1599.173913,2916.000000
std,882.237970,2177.291988
min,214.000000,292.000000
25%,926.250000,1666.250000
50%,1331.000000,2353.000000
75%,2197.500000,3985.000000
max,3443.000000,14272.000000


In [24]:
m = df_lor_data.explore(
     column="inhabitants_total",  # make choropleth based on "BoroName" column
     #scheme="naturalbreaks",  # use mapclassify's natural breaks scheme
     scheme="quantiles",  # use mapclassify's natural breaks scheme
     legend=True, # show legend
     #k=10, # use 10 bins
     legend_kwds=dict(colorbar=False, caption="Inhabitant Density"), # do not use colorbar
     name="inhabitants_total" # name of the layer in the map
)

for i in np.arange(df_lor_data.shape[0]):
    df_circle_plot_1 = gpd.GeoDataFrame({
        "capacity": [df_lor_data.iloc[i]["capacity"] / 100], 
        "geometry": [df_lor_data.iloc[i]["centroid"]]
    })
    df_circle_plot_2 = gpd.GeoDataFrame({
        "vehicles_overall": [df_lor_data.iloc[i]["vehicles_overall"] / 100], 
        "geometry": [df_lor_data.iloc[i]["centroid"]]
    })
    df_circle_plot_1.explore(
         m=m, # pass the map object
         color="red", # use red color on all points
         marker_kwds=dict(radius=df_circle_plot_1.iloc[0]["capacity"], fill=True), # make marker radius 10px with fill
         #tooltip="vehicles_and_lor_ratio_%", # show "name" column in the tooltip
         #tooltip_kwds=dict(labels=False), # do not show column label in the tooltip
         name="capacity" # name of the layer in the map
    )
    df_circle_plot_2.explore(
         m=m, # pass the map object
         color="blue", # use blue color on all points
         marker_kwds=dict(radius=df_circle_plot_2.iloc[0]["vehicles_overall"], fill=True), # make marker radius 10px with fill
         #tooltip="vehicles_and_lor_ratio_%", # show "name" column in the tooltip
         #tooltip_kwds=dict(labels=False), # do not show column label in the tooltip
         name="vehicles_overall" # name of the layer in the map
    )

folium.TileLayer('Stamen Toner', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control

m  # show map